# Restore ELK Snapshot on Measurement Node

## Import

In [1]:
import os
import json
import traceback
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
try:
    fablib = fablib_manager()                
    #fablib.show_config()
except Exception as e:
    print(f"Exception: {e}")
import mflib
print(f"MFLib version  {mflib.__version__} " )
from mflib.mflib import MFLib

MFLib version  1.0.0 


## Slice Info

In [2]:
# Change your slice name and node name
slice_name = "MyMonitoredSlice3"
node1_name = 'Node1'
node2_name = 'Node2'
node3_name = 'Node3'

In [3]:
mf = MFLib(slice_name)

Inititializing slice "MyMonitoredSlice2" for MeasurementFramework.
Found meas node as meas-node at 2620:0:c80:1001:f816:3eff:fe84:2107
Bootstrap status indicates Slice Measurement Framework is ready.


In [4]:
meas_node_name = mf.measurement_node_name
try:
    slice = fablib.get_slice(name=slice_name)
except Exception as e:
    print(f"Fail: {e}")
print (slice)

try:
    meas_node = slice.get_node(name=meas_node_name)
except Exception as e:
    print(f"Fail: {e}")   
#print (meas_node)

-----------  ------------------------------------
Slice Name   MyMonitoredSlice2
Slice ID     0318a65c-1000-4da4-9230-961298c83f59
Slice State  StableOK
Lease End    2023-06-09 18:14:34 +0000
-----------  ------------------------------------


## Restore elk data using existing snapshot

### Check existing snapshot in the repository

In [6]:
repo_name="backup_repository"
cmd = f"curl -X GET 'http://localhost:9200/_cat/snapshots/{repo_name}?v'"
stdout, stderr= meas_node.execute(cmd)

id status start_epoch start_time end_epoch end_time duration indices successful_shards failed_shards total_shards
   % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   114  100   114    0     0  14250      0 --:--:-- --:--:-- --:--:-- 14250
 

### Check the snapshot

In [12]:
snapshot_name="snapshot_test"
cmd = f'curl -X GET "http://localhost:9200/_snapshot/{repo_name}/{snapshot_name}?pretty"'
stdout, stderr= meas_node.execute(cmd)

{
  "error" : {
    "root_cause" : [
      {
        "type" : "repository_exception",
        "reason" : "[backup_repository] could not read repository data from index blob"
      }
    ],
    "type" : "repository_exception",
    "reason" : "[backup_repository] Could not read repository data because the contents of the repository do not match its expected state. This is likely the result of either concurrently modifying the contents of the repository by a process other than this cluster or an issue with the repository's underlying storage. The repository has been disabled to prevent corrupting its contents. To re-enable it and continue using it please remove the repository from the cluster and add it again to make the cluster recover the known state of the repository from its physical contents. The last cluster to write to this repository was [A63ZXCiURES-wcm6MvFiSA] at generation [1].",
    "caused_by" : {
      "type" : "repository_exception",
      "reason" : "[backup_repository] co

In [11]:
# EDIT THIS - choose a list of index files from your local storage to import
local_files = ["./snapshots/elk_backup_2023-06-02_18:23:33.tar"]

output = mf.upload_service_files("elk", local_files)

{'success': True, 'message': 'Service file elk_backup_2023-06-02_18:23:33.tar uploaded successfully.'}


### Restore the data using specific snapshot

In [ ]:
# Need to specify indice names to restore, find in above cell and change below
indice_str= f'"indices": "filebeat-7.13.2-2023.05.29-000001,metricbeat-7.13.2-2023.05.29-000001,packetbeat-7.13.2-2023.05.29-000001",'
repo_name="backup_repository"
snapshot_name = "snapshot_test"
cmd = f'curl -X POST "http://localhost:9200/_snapshot/{repo_name}/{snapshot_name}/_restore?pretty" -H "Content-Type: application/json" -d \'{{{indice_str} "rename_pattern": "(.+)", "rename_replacement": "restored-$1", "include_aliases": false}}\''
#print (cmd)
stdout, stderr= meas_node.execute(cmd)

### Check the progress

In [ ]:
cmd = f'curl -XGET "http://localhost:9200/_cat/recovery?v"'
stdout, stderr= meas_node.execute(cmd)

## Once finished, check indice

In [ ]:
cmd= f'curl -XGET "http://localhost:9200/_cat/indices?pretty"'
stdout, stderr= meas_node.execute(cmd)

## Remove the index life cycle policy

In [ ]:
restored_filebeat_index="restored-filebeat-7.13.2-2023.05.29-000001"
restored_metricbeat_index="restored-metricbeat-7.13.2-2023.05.29-000001"
restored_packetbeat_index = "restored-packetbeat-7.13.2-2023.05.29-000001"
cmd1 = f'curl -X POST "http://localhost:9200/{restored_filebeat_index}/_ilm/remove?pretty"'
cmd2 = f'curl -X POST "http://localhost:9200/{restored_metricbeat_index}/_ilm/remove?pretty"'
cmd3 = f'curl -X POST "http://localhost:9200/{restored_packetbeat_index}/_ilm/remove?pretty"'
stdout, stderr= meas_node.execute(cmd1)
stdout, stderr= meas_node.execute(cmd2)
stdout, stderr= meas_node.execute(cmd3)

## Set up ssh tunnel to view the data

In [ ]:
port = "10050"
cmd=f'ssh -L {port}:localhost:5601 -F ssh_config -i slice_key ubuntu@{meas_node.get_management_ip()}'
print (f"Use [{cmd}] to set up ssh tunnel and then go to http://localhost:{port} to view the data ")

### You will see (restored) filebeat/metricbeat/packetbeat indice(6 total). The restored ones are static with no life cycle policies. The regular filebeat/metricbeat/packetbeat indice will be continuous written to by the beats.   